## Introducción a la Visión Computacional
## Tarea 4
-----------------------
<div align="right">
Fecha de Entrega: Martes 17, Mayo 2022.
</div>

1. Proponer mejoras al sistema diseñado e implementado originalmete.

2. Volver a investigar y averiguar si hay algún algoritmo entrenado o bien si se puede extender el algoritmo utilizado trabajado en la Tarea 2.

3. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) de la comparación con los resultados de la Tarea 2. Notar que pudiera ser que el resultado es inferior al de la Tarea 2, lo importante es el comentario o discusión del por qué se mejoró o empeoraron los nuevos resultados.

# Base de datos de patentes de vehiculos de Chile

Base de datos recopilada por mi, en imagenes de Google, las cuales fueran fotografias de patentes de vehiculos chilenos, con esto me gustaria realizar un posterior analisis, obteniendo las letras y numeros para consultar en [autoseguro.gob.cl](https://www.autoseguro.gob.cl) de carabineros y policia de investigaciones, con el objetivo de verificar si un vehiculo tiene encargo por robo.

En este caso para el pre-procesamiento de imagenes, se aplica el filtro sobel, para poder obtener la deteccion de bordes, con esto podemos tener la capacidad de detectar medianamente bien las letras y numeros de las patentes. Obteniendo estos datos, podria verificar si estos estan con encargo por robo y asi poder informar que el vehiculo fue visto por ultima vez en una locacion especifica

## Definicion funciones para pipeline de procesamiento, conversion y reconocimiento de imagenes
Define funciones intermedias para ser referenciadas desde funcion de ejecucion de pipeline
- obtencion de imagenes desde classpath de proyecto ```get_image_list_from_path(path, extension)```
- pre-procesamiento de imagenes obtenidas en punto anterior con ```save_resized_images_resx_resy```
- procesamiento de imagenes con filtro sobel ```sobel_filter```
- reconocimiento de figuras en imagenes ```detect_vehicle_license_plate```
- reconocimiento optico de caracteres, para llevar a archivo de texto las patentes reconocidas ```ocr_vehicle_license_plate```

In [1]:
import os

import numpy as np
from PIL import Image
from numpy import *
from scipy.ndimage import sobel
from matplotlib import pyplot as plt
import cv2
import time
from pathlib import Path
import shutil
import pytesseract
import glob

# Functiones utilitarias

In [2]:
def stopwatch(sec, function_name):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  print("Time Lapsed = {0}:{1}:{2} in function '{3}'".format(int(hours), int(mins), sec, function_name))

In [3]:
def delete_dir(path_directory):
    dir_path = Path(path_directory)
    if dir_path.exists() and dir_path.is_dir():
        shutil.rmtree(dir_path)

# Funciones de pipeline

In [4]:
'''
Obtiene las imagenes definidas en el classpath del proyecto, con los formatos que se establecen en el segundo parametro
    - path: direccion donde estaran alojadas las imagenes a procesar
    - extension: lista de extensiones que seran tomadas en cuenta para obtencion de imagenes
    - limit: limite de imagenes que obtendra desde el parametro path
'''
def get_image_list_from_path(path, extension, limit = -1):
    l = []
    for ex in extension:
        for f in os.listdir(path):
            if f.endswith(ex):
                l.append(os.path.join(path,f))

    if limit != -1:
        return l[:limit]

    return l

In [5]:
#img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"])

In [6]:
'''
Guarda las imagenes que ya fueron guardadas en el classpath, con una resolucion diferente
    - x: resolucion al eje x
    - y: resolucion al eje y
    - img_list: lista de imagenes que seran procesadas a la resolucion pasada en los parametros x e y
'''
def save_resized_images_resx_resy(x, y, img_list):
    os.makedirs("out/out{}x{}/patentes".format(x, y), exist_ok=True)

    for img in img_list:
        Image.open(img).convert('RGB').resize((x, y))\
            .save("out/out{}x{}/{}".format(x, y, img))

    return get_image_list_from_path("out/out{}x{}/patentes".format(x, y), [".png", ".jpeg"])

In [7]:
#resized_images_512_512 = save_resized_images_resx_resy(512, 512, img_from_path)

In [8]:
'''
De las imagenes que sean pasadas por el parametro 'img_list' (lista de paths), seran convertidas y guardadas con el filtro sobel (reconocimiento de bordes)
    - img_list: lista de imagenes que seran convertidas y guardadas con el filtro sobel
'''
def sobel_filter(img_list):

    os.makedirs("out/out_sobel/patentes", exist_ok=True)

    for img in img_list:
        im = array(Image.open(img).convert('L'), 'f')
        imx = zeros(im.shape)
        sobel(im, 1, imx)

        imy = zeros(im.shape)
        sobel(im, 0, imy)

        magnitude = sqrt(imx**2+imy**2)
        plt.imsave("out/out_sobel/{}".format("/".join(img.split("/")[-2:])), arr = magnitude, cmap = plt.get_cmap('gray'))

    return get_image_list_from_path("out/out_sobel/patentes", [".png", ".jpeg"])

In [9]:
#sobel_filter_images = sobel_filter(resized_images_512_512)

In [10]:
'''
Funcion orientada a reconocimiento de figuras en imagenes pasadas por el parametro 'img_list' (lista de paths)
Las imagenes quedaran con bordes en rojo
    - img_list: lista de imagenes que seran convertidas y guardadas con reconocimiento de figuras
'''
def detect_vehicle_license_plate(img_list):

    os.makedirs("out/out_detection/patentes", exist_ok=True)
    for img_path in img_list:

        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        i = 0
        for contour in contours:
            if i == 0:
                i = 1
                continue
            approx = cv2.approxPolyDP(
                contour, 0.01 * cv2.arcLength(contour, True), True)
            cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
            M = cv2.moments(contour)
            if M['m00'] != 0.0:
                x = int(M['m10']/M['m00'])
                y = int(M['m01']/M['m00'])

        cv2.imwrite('out/out_detection/{}'.format("/".join(img_path.split("/")[-2:])), img)

In [11]:
#detect_vehicle_license_plate(sobel_filter_images)

In [ ]:
# specify path to the license plate images folder as shown below
#path_for_license_plates = "out/out512x512/patentes/*.jpeg"
path_for_license_plates = "patentes/*.jpeg"
list_license_plates = []
predicted_license_plates = []

for path_to_license_plate in glob.glob(path_for_license_plates, recursive=True):
    license_plate_file = path_to_license_plate.split("/")[-1]
    license_plate, _ = os.path.splitext(license_plate_file)

    '''
    Here we append the actual license plate to a list
	'''
    list_license_plates.append(license_plate)

    '''
	Read each license plate image file using openCV
	'''
    img = cv2.imread(path_to_license_plate)

    '''
	We then pass each license plate image file
	to the Tesseract OCR engine using the Python library
	wrapper for it. We get back predicted_result for
	license plate. We append the predicted_result in a
	list and compare it with the original the license plate
	'''
    predicted_result = pytesseract.image_to_string(img, lang='eng',
                                                   config='--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

    filter_predicted_result = "".join(predicted_result.split()).replace(":", "").replace("-", "")
    predicted_license_plates.append(filter_predicted_result)


In [12]:
'''
Funcion de ejecucion de proceso de recoleccion, pre-procesamiento, procesamiento, deteccion

    - limit: define cual es el numero de imagenes con que trabajara en todo el flujo. Por defecto es -1
    - res_x: resolucion al eje x, por defecto 512
    - res_y: resolucion al eje y, por defecto 512

NOTA: Esta funcion generara un reporte de demora en cada uno de las funciones de ejecucion,
como tambien de la demora general en el proceso

'''
def pipeline(limit = -1, res_x = 512, res_y = 512):
    start = time.time()

    elapsed_f_preconditions = None
    elapsed_f_get_image_list_from_path = None
    elapsed_f_save_resized_images_resx_resy = None
    elapsed_f_sobel_filter = None
    elapsed_f_detect_vehicle_license_plate = None

    print("-----------------------")
    print("Pre-conditions")
    try:
        start_preconditions = time.time()
        delete_dir('out')
        end_preconditions = time.time()
        elapsed_f_preconditions = end_preconditions - start_preconditions
        stopwatch(elapsed_f_preconditions, 'pre-conditions block')
    except:
      print("An exception was throw executing pre-conditions block")
    print("-----------------------")

    print("Obtaining images")
    img_from_path = None
    try:
        start_f_get_image_list_from_path = time.time()
        img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"], limit)
        end_f_get_image_list_from_path = time.time()
        elapsed_f_get_image_list_from_path = end_f_get_image_list_from_path - start_f_get_image_list_from_path
        stopwatch(elapsed_f_get_image_list_from_path, 'get_image_list_from_path')
    except:
      print("An exception was throw executing 'get_image_list_from_path' function")
    print("-----------------------")

    print("Resolution adjust to {0}x{1}".format(res_x, res_y))
    resized_images = None
    try:
        start_f_save_resized_images_resx_resy = time.time()
        resized_images = save_resized_images_resx_resy(res_x, res_y, img_from_path)
        end_f_save_resized_images_resx_resy = time.time()
        elapsed_f_save_resized_images_resx_resy = end_f_save_resized_images_resx_resy - start_f_save_resized_images_resx_resy
        stopwatch(elapsed_f_save_resized_images_resx_resy, 'save_resized_images_resx_resy')
    except:
      print("An exception was throw executing 'save_resized_images_resx_resy' function")
    print("-----------------------")

    print("Sobel filter")
    sobel_filter_images = None
    try:
        start_f_sobel_filter = time.time()
        sobel_filter_images = sobel_filter(resized_images)
        end_f_sobel_filter = time.time()
        elapsed_f_sobel_filter = end_f_sobel_filter - start_f_sobel_filter
        stopwatch(elapsed_f_sobel_filter, 'sobel_filter')
    except:
      print("An exception was throw executing 'sobel_filter' function")
    print("-----------------------")

    print("Shape Detection")
    try:
        start_f_detect_vehicle_license_plate = time.time()
        detect_vehicle_license_plate(sobel_filter_images)
        end_f_detect_vehicle_license_plate = time.time()
        elapsed_f_detect_vehicle_license_plate = end_f_detect_vehicle_license_plate - start_f_detect_vehicle_license_plate
        stopwatch(elapsed_f_detect_vehicle_license_plate, 'detect_vehicle_license_plate')
    except:
      print("An exception was throw executing 'detect_vehicle_license_plate' function")
    print("-----------------------")
    print("")

    elapsed = [elapsed_f_preconditions,
               elapsed_f_get_image_list_from_path,
               elapsed_f_save_resized_images_resx_resy,
               elapsed_f_sobel_filter,
               elapsed_f_detect_vehicle_license_plate]

    print("Metrics execution")
    print("Max time execution: {0}".format(np.amax(elapsed)))
    print("Min time execution: {0}".format(np.amin(elapsed)))
    print("Avg time execution: {0}".format(np.average(elapsed)))
    print("Std-Dev(+-) execution: {0}".format(np.std(elapsed)))
    print("")

    end = time.time()
    stopwatch(end - start, 'pipeline')

In [15]:
pipeline()

-----------------------
Pre-conditions
Time Lapsed = 0:0:0.02420783042907715 in function 'pre-conditions block'
-----------------------
Obtaining images
Time Lapsed = 0:0:0.001024007797241211 in function 'get_image_list_from_path'
-----------------------
Resolution adjust to 512x512
Time Lapsed = 0:0:1.2967278957366943 in function 'save_resized_images_resx_resy'
-----------------------
Sobel filter
Time Lapsed = 0:0:1.6773300170898438 in function 'sobel_filter'
-----------------------
Shape Detection
Time Lapsed = 0:0:0.7438671588897705 in function 'detect_vehicle_license_plate'
-----------------------

Metrics execution
Max time execution: 1.6773300170898438
Min time execution: 0.001024007797241211
Avg time execution: 0.7486313819885254
Std-Dev(+-) execution: 0.6703163296597919

Time Lapsed = 0:0:3.7465131282806396 in function 'pipeline'


In [14]:
#pipeline(res_x=256, res_y=256)

-----------------------
Pre-conditions
Time Lapsed = 0:0:0.02186298370361328 in function 'pre-conditions block'
-----------------------
Obtaining images
Time Lapsed = 0:0:0.0008478164672851562 in function 'get_image_list_from_path'
-----------------------
Resolution adjust to 256x256
Time Lapsed = 0:0:0.5141892433166504 in function 'save_resized_images_resx_resy'
-----------------------
Sobel filter
Time Lapsed = 0:0:0.6165659427642822 in function 'sobel_filter'
-----------------------
Shape Detection
Time Lapsed = 0:0:0.31499791145324707 in function 'detect_vehicle_license_plate'
-----------------------

Metrics execution
Max time execution: 0.6165659427642822
Min time execution: 0.0008478164672851562
Avg time execution: 0.29369277954101564
Std-Dev(+-) execution: 0.2501876451848947

Time Lapsed = 0:0:1.4696660041809082 in function 'pipeline'


In [2]:
# Import required packages
import cv2
import pytesseract

# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = 'tesseract'

# Read image from which text needs to be extracted
img = cv2.imread("patentes/6.jpeg")

# Preprocessing the image starts

# Convert the image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size.
# Kernel size increases or decreases the area
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                                 cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]

    # Open the file in append mode
    file = open("recognized.txt", "a")

    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)

    # Appending the text into file
    file.write(text)
    file.write("\n")

    # Close the file
    file.close

In [1]:
# Loading the required python modules
import pytesseract # this is tesseract module
import matplotlib.pyplot as plt
import cv2 # this is opencv module
import glob
import os

In [4]:
# specify path to the license plate images folder as shown below
#path_for_license_plates = "out/out512x512/patentes/*.jpeg"
path_for_license_plates = "patentes/*.jpeg"
list_license_plates = []
predicted_license_plates = []

for path_to_license_plate in glob.glob(path_for_license_plates, recursive = True):

    license_plate_file = path_to_license_plate.split("/")[-1]
    license_plate, _ = os.path.splitext(license_plate_file)

    '''
    Here we append the actual license plate to a list
	'''
    list_license_plates.append(license_plate)

    '''
	Read each license plate image file using openCV
	'''
    img = cv2.imread(path_to_license_plate)

    '''
	We then pass each license plate image file
	to the Tesseract OCR engine using the Python library
	wrapper for it. We get back predicted_result for
	license plate. We append the predicted_result in a
	list and compare it with the original the license plate
	'''
    predicted_result = pytesseract.image_to_string(img, lang ='eng',
    config ='--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

    filter_predicted_result = "".join(predicted_result.split()).replace(":", "").replace("-", "")
    predicted_license_plates.append(filter_predicted_result)


In [3]:
len(predicted_license_plates)

58

In [21]:
predicted_license_plates

['',
 '',
 '',
 'FID',
 'B',
 '',
 '',
 '',
 'B0TH43',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'BKXZ89',
 '',
 '',
 '',
 '',
 '',
 '',
 'RIT',
 'BHGH74',
 '',
 'S',
 '',
 'F',
 'GWKG64',
 '',
 '7',
 '',
 'JPC',
 '',
 'JDTRC78',
 '',
 'B',
 '3A5',
 'A',
 'AVRP',
 'VG8961',
 '',
 'GKSB78',
 '',
 '0475',
 '',
 '',
 '',
 '',
 '3',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'OB7']